In [1]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In [2]:
def create_data_model():
    data = {}
    data['distance_matrix'] = [[0.        ,  9.08150584, 11.52262012,  5.8282368 , 17.64029965],
       [ 9.08150584,  0.        , 19.56945236,  6.977526  , 19.91500572],
       [11.52262012, 19.56945236,  0.        , 13.18741504, 14.06403735],
       [ 5.8282368 ,  6.977526  , 13.18741504,  0.        , 13.72528908],
       [17.64029965, 19.91500572, 14.06403735, 13.72528908,  0.        ]]
    
    data['num_vehicles'] = 1
    data['depot'] = 0
    data['demands'] = [0, 3, 5, 2, 6]
    data['vehicle_capacities'] = []
    data['pickups_deliveries'] = [
        [1, 6],
        [2, 10],
        [4, 3],
        [5, 9],
        [7, 8],
        [15, 11],
        [13, 12],
        [16, 14],
    ]
    data['time_windows'] = [
        (0, 5),   # depot
        (7, 12),  # 1
        (10, 15), # 2
        (16, 18), # 3
        (10, 13), # 4
        (0, 5),   # 5
        (5, 10),  # 6
        (0, 4),   # 7
        (5, 10),  # 8
        (0, 3),   # 9
        (10, 16), # 10
        (10, 15), # 11
        (0, 5),   # 12
        (5, 10),  # 13
        (7, 8),   # 14
        (10, 15), # 15
        (11, 15), # 16
    ]

    
    
    
    
    return data

In [6]:
def main():
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'] )
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_node)
        return data['time_windows'][from_node][to_node]
    
#     def demand_callback(from_index):
#         from_node = manager.IndexToNode(from_index)
#         return data['demands'][from_node]
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
#     demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    
    
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


        
    
    
#     routing.AddDimensionWithVehicleCapacity(
#         demand_callback_index,
#         0,
#         data['vehicle_capacities'],
#         True,
#         'Capacity'
#     )
    
    routing.AddDimension(
        transit_callback_index,
        0,
        3000,
        True,
        "Distance"
    )
    
    
    distance_dimension = routing.GetDimensionOrDie("Distance")
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index) 
        )
        
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index) <= distance_dimension.CumulVar(delivery_index)
        )
        
    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
#     search_parameters.first_solution_strategy = (
#         routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
#     )

    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    
    search_parameters.time_limit.FromSeconds(1)
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("no solution")

In [7]:
def print_solution(data, manager, routing, solution):
    max_routing_distance = 0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}: \n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += '{} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id 
            )
        plan_output += '{}\n '.format(manager.IndexToNode(index))
        plan_output += 'Distance of the Route {}m \n'.format(route_distance)
        print(plan_output)
        max_routing_distance = max(route_distance, max_routing_distance)

    print('Maximum of the route distances: {}m'.format(max_routing_distance))

In [8]:
main()

Route for vehicle 0: 
0 -> 16 -> 14 -> 13 -> 12 -> 0
 Distance of the Route 1780m 

Route for vehicle 1: 
0 -> 5 -> 2 -> 10 -> 9 -> 0
 Distance of the Route 1712m 

Route for vehicle 2: 
0 -> 4 -> 3 -> 15 -> 11 -> 0
 Distance of the Route 2032m 

Route for vehicle 3: 
0 -> 7 -> 1 -> 6 -> 8 -> 0
 Distance of the Route 1780m 

Maximum of the route distances: 2032m
